In [1]:
import sys
import pandas as pd

sys.path.append('../src') 
from util import get_rcode_df
from util import read_csv
from preprocessing import join_meta
from es_client import *

In [3]:
rcode_df = get_rcode_df('../data/road_code.csv')
 
es = ESClient() 
sales_df = es._search_by_aptname('real-estate', "단대푸르지오") 
print (sales_df)

df = join_meta(sales_df, rcode_df)

     date    거래금액  건축년도     년  법정동     아파트   월      일   전용면적   지번   지역코드   층
0  201209  47,500  2012  2012  단대동  단대푸르지오   9   1~10  84.70  100  41131   9
1  201209  33,200  2012  2012  단대동  단대푸르지오   9   1~10  59.72  100  41131   1
2  201209  32,300  2012  2012  단대동  단대푸르지오   9  11~20  59.97  100  41131   1
3  201209  44,800  2012  2012  단대동  단대푸르지오   9  21~30  84.70  100  41131   2
4  201210  47,500  2012  2012  단대동  단대푸르지오  10   1~10  84.70  100  41131  15
5  201305  44,800  2012  2013  단대동  단대푸르지오   5   1~10  84.74  100  41131   2
6  201306  43,900  2012  2013  단대동  단대푸르지오   6  11~20  84.70  100  41131   1
7  201307  43,900  2012  2013  단대동  단대푸르지오   7   1~10  84.70  100  41131   1
8  201309  43,500  2012  2013  단대동  단대푸르지오   9   1~10  84.74  100  41131   2
9  201309  44,800  2012  2013  단대동  단대푸르지오   9  21~30  84.74  100  41131   2


In [10]:
df['평수'] = df['전용면적'].apply(lambda x: x / float(3.30578))
# 60m2이하, 60~85m2, 85m2초과 
labels = ["60m이하", "60-85m2이하", "85m2초과"]
df['규모'] = pd.cut(df['전용면적'], [0, 65, 85, 99999999],labels = labels)
df['건축년도'] = df['건축년도'].astype(str)
df['년'] = df['년'].astype(str)
df['월'] = df['월'].astype(str)
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'], format="%Y%m")
df['거래금액'] = df['거래금액'].apply(lambda x: int(x.replace(',', ''))) 
df['평당거래액'] = df['거래금액'] / df['전용면적']

In [4]:
df.head(10)

,date,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,시,구,도로명,시군구
0,201209,"47,500",2012,2012,단대동,단대푸르지오,9,1~10,84.70,100,41131,9,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
1,201209,"33,200",2012,2012,단대동,단대푸르지오,9,1~10,59.72,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
2,201209,"32,300",2012,2012,단대동,단대푸르지오,9,11~20,59.97,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
3,201209,"44,800",2012,2012,단대동,단대푸르지오,9,21~30,84.70,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
4,201210,"47,500",2012,2012,단대동,단대푸르지오,10,1~10,84.70,100,41131,15,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
5,201305,"44,800",2012,2013,단대동,단대푸르지오,5,1~10,84.74,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
6,201306,"43,900",2012,2013,단대동,단대푸르지오,6,11~20,84.70,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
7,201307,"43,900",2012,2013,단대동,단대푸르지오,7,1~10,84.70,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
8,201309,"43,500",2012,2013,단대동,단대푸르지오,9,1~10,84.74,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구
9,201309,"44,800",2012,2013,단대동,단대푸르지오,9,21~30,84.74,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구


# 전처리 함수

* preprocessing.preprocessing_rent
  * 전세
* preprocessing.preprocessing_trade
  * 매매

In [6]:
from preprocessing import *

In [7]:
preprocessing_trade(df).head(10)

,date,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,시,구,도로명,시군구,평당거래액,평수,규모
0,2012-09-01,47500,2012,2012,단대동,단대푸르지오,9,1~10,84.70,100,41131,9,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구,560.802834,25.621790,60-85m2이하
1,2012-09-01,33200,2012,2012,단대동,단대푸르지오,9,1~10,59.72,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구,555.927662,18.065328,60m이하
2,2012-09-01,32300,2012,2012,단대동,단대푸르지오,9,11~20,59.97,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구,538.602635,18.140953,60m이하
3,2012-09-01,44800,2012,2012,단대동,단대푸르지오,9,21~30,84.70,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구,528.925620,25.621790,60-85m2이하
4,2012-10-01,47500,2012,2012,단대동,단대푸르지오,10,1~10,84.70,100,41131,15,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구,560.802834,25.621790,60-85m2이하
5,2013-05-01,44800,2012,2013,단대동,단대푸르지오,5,1~10,84.74,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구,528.675950,25.633890,60-85m2이하
6,2013-06-01,43900,2012,2013,단대동,단대푸르지오,6,11~20,84.70,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구,518.299882,25.621790,60-85m2이하
7,2013-07-01,43900,2012,2013,단대동,단대푸르지오,7,1~10,84.70,100,41131,1,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구,518.299882,25.621790,60-85m2이하
8,2013-09-01,43500,2012,2013,단대동,단대푸르지오,9,1~10,84.74,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구,513.334907,25.633890,60-85m2이하
9,2013-09-01,44800,2012,2013,단대동,단대푸르지오,9,21~30,84.74,100,41131,2,경기도,성남시 수정구,창업로57번길,경기도 성남시 수정구,528.675950,25.633890,60-85m2이하
